In [1]:
# === Notebook Setup Cell ===
import sys, os

# Add project root (evcs-projects/)
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if project_root not in sys.path:
    sys.path.append(project_root)

# Add scripts folder
scripts_path = os.path.abspath(os.path.join(project_root, "scripts"))
if scripts_path not in sys.path:
    sys.path.append(scripts_path)

print("Python path configured. Last entries:")
print(sys.path[-2:])

# import comparison helper functions (must exist in comparisonscript.py)
from comparisonscript import run_batch_one_pass


Python path configured. Last entries:
['c:\\Users\\asus\\OneDrive\\EV-projects\\evcs-projects', 'c:\\Users\\asus\\OneDrive\\EV-projects\\evcs-projects\\scripts']


In [2]:
# === Two-table notebook cell (updated: includes WEIGHTED) ===

import pandas as pd
import numpy as np
from pathlib import Path

# -----------------------------
# 1) Deterministic pass (MAIN TABLE)
# -----------------------------
df_det = run_batch_one_pass(
    Ns=(20,40,60),
    seeds=(1,),
    policies=("closest_only", "closest_priority", "system_optimum", "uniform"),
    greedy_mode="deterministic",
    max_iter=30,
)

# Pretty MultiIndex header for the main deterministic table (unchanged)
df_main = df_det.copy()
df_main.columns = pd.MultiIndex.from_tuples([
    ("ID", "N"),
    ("ID", "seed"),
    ("ID", "policy"),
    ("Exact", "Score exact"),
    ("Exact", "Time (s)"),
    ("Exact", "Gap (%)"),
    ("Heuristic", "Greedy score"),
    ("Heuristic", "LS score"),
    ("Heuristic", "LS time (s)"),
    ("Heuristic", "SA score"),
    ("Heuristic", "SA time (s)"),
])

# -----------------------------
# 2) RANDOM GREEDY (comparison)
# -----------------------------
df_rand = run_batch_one_pass(
    Ns=(20,40,60),
    seeds=(1,),
    policies=("closest_only", "closest_priority", "system_optimum", "uniform"),
    greedy_mode="random",
    max_iter=30,
)

# -----------------------------
# 3) WEIGHTED GREEDY (W1) — NEW
# -----------------------------
df_wgt = run_batch_one_pass(
    Ns=(20,40,60),
    seeds=(1,),
    policies=("closest_only", "closest_priority", "system_optimum", "uniform"),
    greedy_mode="weighted",  # <-- requires the changes we made in methods.py & comparisonscript.py
    max_iter=30,
)

# -----------------------------
# 4) Build comparison table (Det vs Random vs Weighted)
# -----------------------------
det = df_det.rename(columns={
    "score_greedy": "det_greedy",
    "score_LS":     "det_LS",
    "time_LS":      "det_LS_time",
})
det = det[["N","seed","policy","score_exact","time_exact","det_greedy","det_LS","det_LS_time"]]

rnd = df_rand.rename(columns={
    "score_greedy": "random_greedy",
    "score_LS":     "random_LS",
    "time_LS":      "random_LS_time",
})
rnd = rnd[["N","seed","policy","random_greedy","random_LS","random_LS_time"]]

wgt = df_wgt.rename(columns={
    "score_greedy": "weighted_greedy",
    "score_LS":     "weighted_LS",
    "time_LS":      "weighted_LS_time",
})
wgt = wgt[["N","seed","policy","weighted_greedy","weighted_LS","weighted_LS_time"]]

# Merge into one row per (N, seed, policy)
df_compare = det.merge(rnd, on=["N","seed","policy"], how="inner").merge(
    wgt, on=["N","seed","policy"], how="inner"
)

# Optional: show how close LS is to exact and how weighted compares to det/random
# Positive is better; values are percentages of exact
df_compare["Det vs Exact (%)"]      = 100 * (df_compare["det_LS"]      / df_compare["score_exact"] - 1.0)
df_compare["Random vs Exact (%)"]   = 100 * (df_compare["random_LS"]   / df_compare["score_exact"] - 1.0)
df_compare["Weighted vs Exact (%)"] = 100 * (df_compare["weighted_LS"] / df_compare["score_exact"] - 1.0)

# Direct deltas vs deterministic (in % of exact) to highlight the "weighted effect"
df_compare["(Weighted-Det) / Exact (%)"] = 100 * (df_compare["weighted_LS"] - df_compare["det_LS"]) / df_compare["score_exact"]
df_compare["(Weighted-Rand) / Exact (%)"] = 100 * (df_compare["weighted_LS"] - df_compare["random_LS"]) / df_compare["score_exact"]

# Reorder columns for readability
df_compare = df_compare[[
    "N","seed","policy",
    "score_exact","time_exact",
    "det_greedy","det_LS","det_LS_time",
    "random_greedy","random_LS","random_LS_time",
    "weighted_greedy","weighted_LS","weighted_LS_time",
    "Det vs Exact (%)","Random vs Exact (%)","Weighted vs Exact (%)",
    "(Weighted-Det) / Exact (%)","(Weighted-Rand) / Exact (%)",
]]

# -----------------------------
# 5) Export both tables to ONE Excel (now with a 3-mode sheet)
# -----------------------------
Path("results").mkdir(exist_ok=True)
out_xlsx = "results/two_tables_summary.xlsx"

# Make flat copies ONLY for saving to Excel
df_main_flat = df_main.copy()
df_main_flat.columns = ["_".join(col) for col in df_main_flat.columns]

df_compare_flat = df_compare.copy()

with pd.ExcelWriter(out_xlsx, engine="xlsxwriter") as writer:
    df_main_flat.to_excel(writer, sheet_name="Main (deterministic)", index=False)
    df_compare_flat.to_excel(writer, sheet_name="Greedy Comparison (3 modes)", index=False)

print(f"✅ Saved two tables to {out_xlsx}")

# -----------------------------
# 6) Display pretty versions in the notebook
# -----------------------------
display(df_main)     # pretty MultiIndex header (deterministic)
display(df_compare)  # comparison across deterministic, random, weighted



📌 N=20 | Policy=closest_only | greedy_mode=deterministic
  🔹 Exact solve ...
→ Added 'closest_only' constraints.
    ✅ Exact OPTIMAL: 60.000 (t=0.07s)
  🔹 Greedy initialization ...
→ Added 'closest_only' constraints.
     → Using SMART deterministic greedy
     Greedy score = 52.178
  🔹 Local Search ...

🔵 Local Search start (closest_only), initial obj=52.178
Iter 01: open/close 10->13 → 54.017
↪️ Switching to next neighborhood: MERGE
↪️ Switching to next neighborhood: SHIFT
🔚 No improvement in any neighborhood. Stopping at iter 16.
✅ Final objective (closest_only) = 54.017
     LS score = 54.017 (t=0.62s)
  🔹 Simulated Annealing ...

🔥 Simulated Annealing start (closest_only) | initial obj=54.017
Iter 029: SA improved → 55.343 (T=0.9500)
✅ Final SA objective (closest_only) = 55.343
     SA score = 55.343 (t=0.23s)

📌 N=20 | Policy=closest_priority | greedy_mode=deterministic
  🔹 Exact solve ...
    ✅ Exact OPTIMAL: 59.992 (t=0.04s)
  🔹 Greedy initialization ...
     → Using SMART det

ID                              Exact                           Heuristic  \
     N seed            policy Score exact    Time (s)    Gap (%) Greedy score   
0   20    1      closest_only   60.000000    0.066354   9.971194    52.177969   
1   20    1  closest_priority   59.991538    0.040744   9.976297    52.170046   
2   20    1    system_optimum   57.471534    0.051227  10.652534    49.885673   
3   20    1           uniform   17.515237    0.023526   0.000000    17.515237   
4   40    1      closest_only  110.611676    4.596023   1.030031   105.371702   
5   40    1  closest_priority  112.274105    0.119700   6.167122   105.350024   
6   40    1    system_optimum  106.100835    0.123567   4.636979   101.180962   
7   40    1           uniform   19.322332    0.049802   0.000000    19.322332   
8   60    1      closest_only  179.388645  120.202734   1.144893   168.521426   
9   60    1  closest_priority  179.971064    0.114191   1.715610   166.441207   
10  60    1    system_optimum  171.815212    2.015215   2.932643   159.929734   
11  60    1           uniform   23.983676    0.075866   0.000000    23.983676   

                                                    
      LS score LS time (s)    SA score SA time (s)  
0    54.017284    0.622692   55.343494    0.229407  
1    54.006604    0.707092   55.330003    0.242327  
2    51.349360    0.851559   51.349360    0.299402  
3    17.515237    1.001320   17.515237    0.274240  
4   109.472342    8.514918  109.472342    1.000844  
5   105.350024    8.635857  109.444113    1.182322  
6   101.180962    8.386692  102.843120    1.153376  
7    19.322332   10.963775   19.322332    1.097245  
8   177.334837   36.873528  177.334837    2.040852  
9   176.883463   45.911198  176.883463    2.178654  
10  166.776484   36.100121  166.776484    2.197980  
11   23.983676   38.790953   23.983676    2.283999

,N,seed,policy,score_exact,time_exact,det_greedy,det_LS,det_LS_time,random_greedy,random_LS,random_LS_time,weighted_greedy,weighted_LS,weighted_LS_time,Det vs Exact (%),Random vs Exact (%),Weighted vs Exact (%),(Weighted-Det) / Exact (%),(Weighted-Rand) / Exact (%)
0,20,1,closest_only,60.000000,0.066354,52.177969,54.017284,0.622692,48.498638,55.343494,0.864198,40.098565,50.991637,0.623877,-9.971194,-7.760843,-15.013939,-5.042745,-7.253096
1,20,1,closest_priority,59.991538,0.040744,52.170046,54.006604,0.707092,31.948597,55.327781,1.012300,44.510433,55.330003,0.737301,-9.976297,-7.774025,-7.770321,2.205976,0.003705
2,20,1,system_optimum,57.471534,0.051227,49.885673,51.349360,0.851559,31.046020,52.294424,1.007797,48.237506,51.349360,0.973209,-10.652534,-9.008130,-10.652534,0.000000,-1.644404
3,20,1,uniform,17.515237,0.023526,17.515237,17.515237,1.001320,13.733325,17.515237,1.125628,45.062182,17.515237,1.351915,0.000000,0.000000,0.000000,0.000000,0.000000
4,40,1,closest_only,110.611676,4.596023,105.371702,109.472342,8.514918,96.642093,108.808503,8.210288,89.365344,110.611676,7.044159,-1.030031,-1.630183,0.000000,1.030031,1.630183
5,40,1,closest_priority,112.274105,0.119700,105.350024,105.350024,8.635857,36.237935,108.914544,7.928292,100.552179,106.944287,9.161608,-6.167122,-2.992285,-4.747148,1.419974,-1.754863
6,40,1,system_optimum,106.100835,0.123567,101.180962,101.180962,8.386692,34.429814,103.397855,11.131215,73.924843,102.833799,7.968962,-4.636979,-2.547558,-3.079180,1.557798,-0.531622
7,40,1,uniform,19.322332,0.049802,19.322332,19.322332,10.963775,16.581530,19.322332,11.881527,88.855491,19.322332,9.246369,0.000000,0.000000,0.000000,0.000000,0.000000
8,60,1,closest_only,179.388645,120.202734,168.521426,177.334837,36.873528,126.696077,173.540714,34.951157,144.404069,176.153271,30.675957,-1.144893,-3.259923,-1.803556,-0.658663,1.456367
9,60,1,closest_priority,179.971064,0.114191,166.441207,176.883463,45.911198,85.167426,175.701553,36.066013,152.504928,173.774736,42.665536,-1.715610,-2.372332,-3.442958,-1.727348,-1.070626
